In [1]:
import spotipy
import urllib.request
import random
import requests
import nltk
import webbrowser

In [2]:
with open("news_key.txt", "r") as file:
    key = file.read()

In [3]:
url = "https://newsapi.org/v2/top-headlines?country=us&category=entertainment&apiKey=" + key

In [4]:
url

'https://newsapi.org/v2/top-headlines?country=us&category=entertainment&apiKey=19e7b8a1be8f48a790f4655dde3660e4'

In [5]:
# requesting the information
request = urllib.request.Request(url)
# the information is coming back to me
response = urllib.request.urlopen(request)

In [6]:
response

In [7]:
response.status

200